In [87]:
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import mysql.connector
import os
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [85]:
service = Service(executable_path='/Users/yuna/Desktop/BAX-422 DDR/chromedriver-mac-arm64/chromedriver')
driver = WebDriver(service=service)

We will first attempt to extract all comments for 10 videos from each of the keyword. We have 5 keyword searching on Youtube, so that will give us all comments for 50 videos under 1 genre, 200 videos in total

In [93]:
# Extract comments
def scrape_comments(driver, genre, keyword, video_id):
    filename = f"{genre}_{keyword}_{video_id}.html"
    
    output_dir = '/Users/yuna/Desktop/BAX-422 DDR/Final project/youtube_htmls_scrolled'
    filepath = os.path.join(output_dir, filename)
    
    # Check if the file already exists
    if os.path.exists(filepath):
        print(f"File for {video_id} already exists. Skipping.")
        return False  # Skip scrap if file already exists
    
    # Scroll down to load more comments
    SCROLL_PAUSE_TIME = 3

    # Get the comments section element
    comments_section = driver.find_element(By.CSS_SELECTOR, "#comments")

    # Define the scroll function
    def scroll_down():
        # Scroll down by a certain amount
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(2)  

    # Scroll down to load more comments
    prev_scroll_position = None
    scroll_counter = 0 
    while scroll_counter < 10:  # Limit to 10 scrolls
        # Scroll down
        scroll_down()

        # Check if new comments are loaded by looking for a specific element that appears when new comments are loaded
        try:
            new_comments_element = driver.find_element(By.XPATH, '//*[@id="continuation"]')
        except NoSuchElementException:
            # If new comments are not loaded, you've reached the end of the comments section
            break

        # Increment the scroll counter
        scroll_counter += 1

    # Extract comments after scrolling
    comments_html = comments_section.get_attribute('innerHTML')
    with open(filepath, "w", encoding='utf-8') as file:
        file.write("<html><head><title>YouTube Comments</title></head><body>")
        file.write(comments_html)
        file.write("</body></html>")
    
    return True  # comments successfully scraped


driver_youtube = webdriver.Chrome()

# Genres and corresponding keywords
genres = {
    'Sports': ['Football compilations', 'hockey compilations', 'cricket compilations', 'athletics top moments', 'tennis top moments'],
    'Documentary': ['Nature documentaries', 'History documentaries', 'Science documentaries', 'True crime documentaries', 'Technology documentaries'],
    'Movie': ['The Ultimate 2024 Oscar Trailers', 'Action movie trailers', '21st century best movies', 'Marvel movie trailers', 'Netflix original series'],
    'Lo-fi': ['Lo-fi hip-hop beats', 'Lo-fi chillhop mixes', 'Lo-fi study music', 'Lo-fi jazz playlists', 'Lo-fi ambient music']
}

for genre, keywords in genres.items():
    for keyword in keywords:
        url = f'https://www.youtube.com/results?search_query={keyword}'
        driver_youtube.get(url)
        time.sleep(3)

        videos_processed = 0
        video_index = 0
        
        while videos_processed < 50:
            videos = driver_youtube.find_elements(By.CSS_SELECTOR, 'a#video-title')
            if video_index >= len(videos):
                print("Ran out of videos to process.")
                break

            video = videos[video_index]
            time.sleep(2)
            video.click()
            time.sleep(3)

            video_url = driver_youtube.current_url

            # Skip shorts, we only want
            if "/shorts/" in video_url:
                driver_youtube.back()
                time.sleep(3)
                video_index += 1
                continue

            video_id = video_url.split('watch?v=')[-1]
            
            # Attempt to scrape comments and check if the operation was skipped
            scraped = scrape_comments(driver_youtube, genre, keyword, video_id)
            if scraped:
               1 videos_processed += 1

            driver_youtube.back()
            time.sleep(3)
            video_index += 1

driver_youtube.quit()


File for Qp5vnQ331AY already exists. Skipping.
File for nU7fK3UUepU already exists. Skipping.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.
Ran out of videos to process.


In [ ]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [95]:
# Mapping genre and category id
genre_to_category_id = {
    'Sports': 1,
    'Documentary': 2,
    'Movie': 3,
    'Lo-fi': 4,
}

# Database configuration and connection setup
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Dmsdk9429!',
    'database': 'youtube_data'
}

conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

html_files_dir = '/Users/yuna/Desktop/BAX-422 DDR/Final project/youtube_htmls_scrolled'

# Putting comment, genre, keyword, and video_id into the database
def process_html_file(file_path, genre, keyword, video_id):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file.read(), 'html.parser')
    
    comments = soup.find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
    category_id = genre_to_category_id.get(genre, 0)  # Get category ID based on genre
    
    for comment in comments:
        comment_text = comment.find('yt-formatted-string', id='content-text').text if comment.find('yt-formatted-string', id='content-text') else "No Comment Text"
        likes_span = comment.find('span', id='vote-count-middle')
        likes = likes_span.text.strip() if likes_span else "0"
        likes = int(likes.replace('K', '000').replace('M', '000000').replace('.', '')) if 'K' in likes or 'M' in likes else int(likes)

        try:
            cursor.execute('''
            INSERT INTO youtube_comments (genre, keyword, category_id, video_id, comment, likes) VALUES (%s, %s, %s, %s, %s, %s)
            ''', (genre, keyword, category_id, video_id, comment_text, likes))
        except mysql.connector.Error as err:
            print("Something went wrong: {}".format(err))


# Loop through the html files and process them
for filename in os.listdir(html_files_dir):
    if filename.endswith('.html'):
        parts = filename[:-5].split('_')
        genre = parts[0]
        keyword = '_'.join(parts[1:-1])  # Extract keyword from the filename
        video_id = parts[-1]  # Extract video_id from the filename
        print(f"Processing: Genre - {genre}, Keyword - {keyword}, Video ID - {video_id}")
        
        file_path = os.path.join(html_files_dir, filename)
        process_html_file(file_path, genre, keyword, video_id)

conn.commit()
cursor.close()
conn.close()


Processing: Genre - Lo-fi, Keyword - Lo-fi study music_EtD7, Video ID - 8kCMHA
Processing: Genre - Sports, Keyword - tennis top moments, Video ID - syg1xTNT9b0
Processing: Genre - Sports, Keyword - Football compilations, Video ID - LQXLEW8dMoc
Processing: Genre - Movie, Keyword - Action movie trailers, Video ID - Vw7hIfI-GC0
Processing: Genre - Movie, Keyword - 21st century best movies, Video ID - ioTU0GY3z-c
Processing: Genre - Movie, Keyword - Action movie trailers, Video ID - aDyQxtg0V2w
Processing: Genre - Movie, Keyword - The Ultimate 2024 Oscar Trailers, Video ID - GjLYsSDiXFs
Processing: Genre - Movie, Keyword - Marvel movie trailers, Video ID - ObnVqcD1qC0
Processing: Genre - Sports, Keyword - Football compilations, Video ID - 4Ryf7gIWdtc
Processing: Genre - Sports, Keyword - Football compilations, Video ID - g0LPYf61l1A
Processing: Genre - Sports, Keyword - athletics top moments, Video ID - 3TspXGC8LW4
Processing: Genre - Movie, Keyword - The Ultimate 2024 Oscar Trailers, Vide

In [102]:
import csv

input = '/Users/yuna/Desktop/BAX-422 DDR/Final project/youtube_data_scrolled.csv'
output = '/Users/yuna/Desktop/BAX-422 DDR/Final project/youtube_data_final.csv'



In [103]:
# checking how many rows are in the csv file
with open(input, 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)
    print(len(rows))

24394


In [107]:
# Read input CSV file and filter out empty rows
with open(input, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    rows = [row for row in reader if len(row) >= 5 and row[4]]   # Filter out empty rows

# Write filtered data to output CSV file
with open(output, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(rows)

In [108]:
# checking how many rows are in the csv file
with open(output, 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)
    print(len(rows))

23553
